In [1]:
import yfinance as yf
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, Select, CustomJS,RangeTool
#from bokeh.palettes import Spectral3
#from bokeh.palettes import Spectral6
from bokeh.layouts import gridplot,column
from bokeh.palettes import Category20c
from bokeh.transform import cumsum
from datetime import date, datetime
from dateutil.relativedelta import relativedelta

from fbprophet import Prophet

output_file('StockAnalysis_Rebecca_Hobeika.html')

stocks = yf.Tickers('MSFT AAPL GOOGL AMZN IBM')

#get stock info
#print(stocks.tickers.MSFT)

#get historical market data
msftStockHistory1Year = stocks.tickers.MSFT.history(period="1y")
aaplStockHistory1Year = stocks.tickers.AAPL.history(period="1y")
googStockHistory1Year = stocks.tickers.GOOGL.history(period="1y")
amznStockHistory1Year = stocks.tickers.AMZN.history(period="1y")
ibmStockHistory1Year = stocks.tickers.IBM.history(period="1y")

# msftStockCashflow1Year = stocks.tickers.MSFT.actions
msftStockHistoryDF = pd.DataFrame(msftStockHistory1Year)
aaplStockHistoryDF = pd.DataFrame(aaplStockHistory1Year)
googStockHistoryDF = pd.DataFrame(googStockHistory1Year)
amznStockHistoryDF = pd.DataFrame(amznStockHistory1Year)
ibmStockHistoryDF  = pd.DataFrame(ibmStockHistory1Year)

msftStockHistoryDF = msftStockHistoryDF.reset_index()
aaplStockHistoryDF = aaplStockHistoryDF.reset_index()
googStockHistoryDF = googStockHistoryDF.reset_index()
amznStockHistoryDF = amznStockHistoryDF.reset_index()
ibmStockHistoryDF = ibmStockHistoryDF.reset_index()

#dates = np.array(msftStockHistory1Year["Date"], dtype=np.datetime64)

#print(msftStockHistoryDF)
print("** GOT DATA **")
hover = HoverTool(
    tooltips=[
        ("Volume", "@Volume"),
        ("Date", "@Date{%F}"),
    ],
    formatters={'@Date': 'datetime'}
)

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select,help"
p = figure(tools =[TOOLS,hover],x_axis_type='datetime',title="Volume Trend By Stock Type",y_axis_label = 'Volume',x_axis_label = 'Date',plot_width=1000, x_range=(msftStockHistoryDF['Date'].min(), msftStockHistoryDF['Date'].max())) 

cdsMSFT = ColumnDataSource(msftStockHistoryDF)
cdsAAPL = ColumnDataSource(aaplStockHistoryDF)
cdsGOOG = ColumnDataSource(googStockHistoryDF)
cdsAMZN = ColumnDataSource(amznStockHistoryDF)
cdsIBM = ColumnDataSource(ibmStockHistoryDF)

p.line(x='Date', y='Volume', line_width=2, source=ColumnDataSource(msftStockHistoryDF), color="green", legend='Microsoft')
p.line(x='Date', y='Volume', line_width=2, source=ColumnDataSource(aaplStockHistoryDF), color="orange", legend='Apple')
p.line(x='Date', y='Volume', line_width=2, source=ColumnDataSource(googStockHistoryDF), color="red", legend='Google')
p.line(x='Date', y='Volume', line_width=2, source=ColumnDataSource(amznStockHistoryDF), color="yellow", legend='Amazon')
p.line(x='Date', y='Volume', line_width=2, source=ColumnDataSource(ibmStockHistoryDF), color="blue", legend='IBM') 

p.legend.location = "top_left"
p.legend.click_policy = "hide"

select = figure(title="Drag the middle and edges of the selection box to change the range above",
                plot_height=100, plot_width=1000, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

range_tool = RangeTool(x_range=p.x_range)
range_tool.overlay.fill_color = "red"
range_tool.overlay.fill_alpha = 0.2

select.line('Date', 'Close', source=ColumnDataSource(msftStockHistoryDF))
select.ygrid.grid_line_color = None
select.add_tools(range_tool)
select.toolbar.active_multi = range_tool

inc = msftStockHistoryDF.Close > msftStockHistoryDF.Open
dec = msftStockHistoryDF.Open > msftStockHistoryDF.Close
day = 24*60*60*1000
w = 1*day # 1 day in ms

p2 = figure(x_axis_type="datetime", tools =[TOOLS], plot_width=1000, title = "Microsoft Daily Monitor")
#p2.xaxis.major_label_orientation = pi/4
p2.grid.grid_line_alpha=0.3

p2.segment(msftStockHistoryDF['Date'], msftStockHistoryDF.High, msftStockHistoryDF.Date, msftStockHistoryDF.Low, color="black")
p2.vbar(msftStockHistoryDF.Date[inc], w, msftStockHistoryDF.Open[inc], msftStockHistoryDF.Close[inc], fill_color="#D5E1DD", line_color="black")
p2.vbar(msftStockHistoryDF.Date[dec], w, msftStockHistoryDF.Open[dec], msftStockHistoryDF.Close[dec], fill_color="#F2583E", line_color="black")

inc1 = aaplStockHistoryDF.Close > aaplStockHistoryDF.Open
dec1 = aaplStockHistoryDF.Open > aaplStockHistoryDF.Close

p3 = figure(x_axis_type="datetime", tools =[TOOLS], plot_width=1000, title = "Apple Daily Monitor")
#p3.xaxis.major_label_orientation = pi/4
p3.grid.grid_line_alpha=0.3

p3.segment(aaplStockHistoryDF.Date, aaplStockHistoryDF.High, aaplStockHistoryDF.Date, aaplStockHistoryDF.Low, color="black")
p3.vbar(aaplStockHistoryDF.Date[inc1], w, aaplStockHistoryDF.Open[inc1], aaplStockHistoryDF.Close[inc1], fill_color="#D5E1DD", line_color="black")
p3.vbar(aaplStockHistoryDF.Date[dec1], w, aaplStockHistoryDF.Open[dec1], aaplStockHistoryDF.Close[dec1], fill_color="#F2583E", line_color="black")

inc2 = googStockHistoryDF.Close > googStockHistoryDF.Open
dec2 = googStockHistoryDF.Open > googStockHistoryDF.Close

p4 = figure(x_axis_type="datetime", tools =[TOOLS], plot_width=1000, title = "Google Daily Monitor")
#p4.xaxis.major_label_orientation = pi/4
p4.grid.grid_line_alpha=0.3

p4.segment(googStockHistoryDF.Date, googStockHistoryDF.High, googStockHistoryDF.Date, googStockHistoryDF.Low, color="black")
p4.vbar(googStockHistoryDF.Date[inc2], w, googStockHistoryDF.Open[inc2], googStockHistoryDF.Close[inc2], fill_color="#D5E1DD", line_color="black")
p4.vbar(googStockHistoryDF.Date[dec2], w, googStockHistoryDF.Open[dec2], googStockHistoryDF.Close[dec2], fill_color="#F2583E", line_color="black")

inc3 = amznStockHistoryDF.Close > amznStockHistoryDF.Open
dec3 = amznStockHistoryDF.Open > amznStockHistoryDF.Close

p5 = figure(x_axis_type="datetime", tools =[TOOLS], plot_width=1000, title = "Amazon Daily Monitor")
#p5.xaxis.major_label_orientation = pi/4
p5.grid.grid_line_alpha=0.3

p5.segment(amznStockHistoryDF.Date, amznStockHistoryDF.High, amznStockHistoryDF.Date, amznStockHistoryDF.Low, color="black")
p5.vbar(amznStockHistoryDF.Date[inc3], w, amznStockHistoryDF.Open[inc3], amznStockHistoryDF.Close[inc3], fill_color="#D5E1DD", line_color="black")
p5.vbar(amznStockHistoryDF.Date[dec3], w, amznStockHistoryDF.Open[dec3], amznStockHistoryDF.Close[dec3], fill_color="#F2583E", line_color="black")

inc4 = ibmStockHistoryDF.Close > ibmStockHistoryDF.Open
dec4 = ibmStockHistoryDF.Open > ibmStockHistoryDF.Close

p6 = figure(x_axis_type="datetime", tools =[TOOLS], plot_width=1000, title = "IBM Daily Monitor")
#p6.xaxis.major_label_orientation = pi/4
p6.grid.grid_line_alpha=0.3

p6.segment(ibmStockHistoryDF.Date, ibmStockHistoryDF.High, ibmStockHistoryDF.Date, ibmStockHistoryDF.Low, color="black")
p6.vbar(ibmStockHistoryDF.Date[inc4], w, ibmStockHistoryDF.Open[inc4], ibmStockHistoryDF.Close[inc4], fill_color="#D5E1DD", line_color="black")
p6.vbar(ibmStockHistoryDF.Date[dec4], w, ibmStockHistoryDF.Open[dec4], ibmStockHistoryDF.Close[dec4], fill_color="#F2583E", line_color="black")



#######


model = Prophet()
model.fit(msftStockHistoryDF.reset_index().rename(columns={'Date':'ds', 'Volume':'y'}))
future = model.make_future_dataframe(periods=365) #forecasting for 1 year from now.
msftForecastDF = model.predict(future)
msftForecast = figure(title="Microsoft Forecast",
                plot_height=500, plot_width=1000, 
                x_range=(msftForecastDF['ds'].min(), msftForecastDF['ds'].max()),
                x_axis_type="datetime",
                tools=TOOLS)
msftForecast.line('ds', 'yhat', source=ColumnDataSource(msftForecastDF), color="red", legend='Forecast')
msftForecast.line('Date', 'Volume', source=ColumnDataSource(msftStockHistoryDF), color="blue", legend='Actual')

model = Prophet()
model.fit(aaplStockHistoryDF.reset_index().rename(columns={'Date':'ds', 'Volume':'y'}))
future = model.make_future_dataframe(periods=365) #forecasting for 1 year from now.
aaplForecastDF = model.predict(future)
aaplForecast = figure(title="Apple Forecast",
                plot_height=500, plot_width=1000, 
                x_range=(aaplForecastDF['ds'].min(), aaplForecastDF['ds'].max()),
                x_axis_type="datetime",
                tools=TOOLS)
aaplForecast.line('ds', 'yhat', source=ColumnDataSource(aaplForecastDF), color="red", legend='Forecast')
aaplForecast.line('Date', 'Volume', source=ColumnDataSource(aaplStockHistoryDF), color="blue", legend='Actual')


model = Prophet()
model.fit(googStockHistoryDF.reset_index().rename(columns={'Date':'ds', 'Volume':'y'}))
future = model.make_future_dataframe(periods=365) #forecasting for 1 year from now.
googForecastDF = model.predict(future)
googForecast = figure(title="Google Volume Forecast",
                plot_height=500, plot_width=1000, 
                x_range=(googForecastDF['ds'].min(), googForecastDF['ds'].max()),
                x_axis_type="datetime",
                tools=TOOLS)
googForecast.line('ds', 'yhat', source=ColumnDataSource(googForecastDF), color="red", legend='Forecast')
googForecast.line('Date', 'Volume', source=ColumnDataSource(googStockHistoryDF), color="blue", legend='Actual')


model = Prophet()
model.fit(amznStockHistoryDF.reset_index().rename(columns={'Date':'ds', 'Volume':'y'}))
future = model.make_future_dataframe(periods=365) #forecasting for 1 year from now.
amznForecastDF = model.predict(future)
amznForecast = figure(title="Amazon Volume Forecast",
                plot_height=500, plot_width=1000, 
                x_range=(amznForecastDF['ds'].min(), amznForecastDF['ds'].max()),
                x_axis_type="datetime",
                tools=TOOLS)
amznForecast.line('ds', 'yhat', source=ColumnDataSource(amznForecastDF), color="red", legend='Forecast')
amznForecast.line('Date', 'Volume', source=ColumnDataSource(amznStockHistoryDF), color="blue", legend='Actual')


model = Prophet()
model.fit(ibmStockHistoryDF.reset_index().rename(columns={'Date':'ds', 'Volume':'y'}))
future = model.make_future_dataframe(periods=365) #forecasting for 1 year from now.
ibmForecastDF = model.predict(future)
ibmForecast = figure(title="IBM Volume Forecast",
                plot_height=500, plot_width=1000, 
                x_range=(ibmForecastDF['ds'].min(), ibmForecastDF['ds'].max()),
                x_axis_type="datetime",
                tools=TOOLS)
ibmForecast.line('ds', 'yhat', source=ColumnDataSource(ibmForecastDF), color="red", legend='Forecast')
ibmForecast.line('Date', 'Volume', source=ColumnDataSource(ibmStockHistoryDF), color="blue", legend='Actual')


#######

# make a grid

grid = column(p, select, msftForecast, aaplForecast,googForecast,amznForecast,ibmForecast, p2, p3,p4,p5,p6, sizing_mode="stretch_width")
show(grid)


print("** PUBLISHING GRAPH **")



Importing plotly failed. Interactive plots will not work.


** GOT DATA **


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

** PUBLISHING GRAPH **
